# 07-09 API(Keras Functional API) 케라스의 함수형


출처:  https://wikidocs.net/38861

Sequential API는 여러층을 공유하거나 다양한 종류의 입력과 출력을 사용하는 등의 복잡한 모델을 만드는 일에는 한계가 있습니다. 

-> 이번에는 더욱 복잡한 모델을 생성할 수 있는 방식인 Functional API(함수형 API)에 대해서 알아봅니다.

## 1. Sequential API로 만든 모델


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(3, input_dim=4, activation='softmax'))

## 2. Functional API로 만든 모델

1) 전결합 피드 포워드 신경망(Fully-connected FFNN)



In [2]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [3]:
inputs = Input(shape=(10,))
# 위의 코드는 10개의 입력을 받는 입력층을 보여줍니다

In [4]:
inputs = Input(shape=(10,))
hidden1 = Dense(64, activation='relu')(inputs)  # <- 새로 추가
hidden2 = Dense(64, activation='relu')(hidden1) # <- 새로 추가
output = Dense(1, activation='sigmoid')(hidden2) # <- 새로 추가

In [5]:
inputs = Input(shape=(10,))
hidden1 = Dense(64, activation='relu')(inputs)
hidden2 = Dense(64, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)
model = Model(inputs=inputs, outputs=output) # <- 새로 추가

지금까지의 내용을 정리하면 다음과 같습니다.

# Input() 함수에 입력의 크기를 정의합니다.
# 이전층을 다음층 함수의 입력으로 사용하고, 변수에 할당합니다.
# Model() 함수에 입력과 출력을 정의합니다.
# 이를 model로 저장하면 sequential
# API를 사용할 때와 마찬가지로 model.compile, model.fit 등을 사용 가능합니다.

In [6]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(data, labels)

In [7]:
# 이번에는 변수명을 달리해서 FFNN을 만들어보겠습니다.

inputs = Input(shape=(10,))
x = Dense(8, activation="relu")(inputs)
x = Dense(4, activation="relu")(x)
x = Dense(1, activation="linear")(x)
model = Model(inputs, x)

### 2) 선형 회귀(Linear Regression)

앞서 ( https://wikidocs.net/111472 ) Sequential API로 구현했던 선형 회귀를 Functional API로 구현해봅시다.

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
​
X = [1, 2, 3, 4, 5, 6, 7, 8, 9] # 공부하는 시간
y = [11, 22, 33, 44, 53, 66, 77, 87, 95] # 각 공부하는 시간에 맵핑되는 성적
​
inputs = Input(shape=(1,))
output = Dense(1, activation='linear')(inputs)
linear_model = Model(inputs, output)
​
sgd = optimizers.SGD(lr=0.01)
​
linear_model.compile(optimizer=sgd, loss='mse', metrics=['mse'])
linear_model.fit(X, y, epochs=300)

### 3) 로지스틱 회귀(Logistic Regression)


from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(3,))
output = Dense(1, activation='sigmoid')(inputs)
logistic_model = Model(inputs, output)

### 4) 다중 입력을 받는 모델(model that accepts multiple inputs)

functional API를 사용하면 아래와 같이 다중 입력과 다중 출력을 가지는 모델도 만들 수 있습니다.

# 최종 완성된 다중 입력, 다중 출력 모델의 예
model = Model(inputs=[a1, a2], outputs=[b1, b2, b3])

from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model

두 개의 입력층을 정의

inputA = Input(shape=(64,))

inputB = Input(shape=(128,))

첫번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의

x = Dense(16, activation="relu")(inputA)

x = Dense(8, activation="relu")(x)

x = Model(inputs=inputA, outputs=x)


두번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의

y = Dense(64, activation="relu")(inputB)

y = Dense(32, activation="relu")(y)

y = Dense(8, activation="relu")(y)

y = Model(inputs=inputB, outputs=y)

두개의 인공 신경망의 출력을 연결(concatenate)

result = concatenate([x.output, y.output])

z = Dense(2, activation="relu")(result)

z = Dense(1, activation="linear")(z)

model = Model(inputs=[x.input, y.input], outputs=z)



### 5) RNN(Recurrence Neural Network) 은닉층 사용하기

이번에는 RNN 은닉층을 가지는 모델을 설계해봅시다. 여기서는 하나의 특성(feature)에 50개의 시점(time-step)을 입력으로 받는 모델을 설계해보겠습니다. RNN에 대한 구체적인 내용은 다음 챕터인 RNN 챕터에서 배웁니다.

from tensorflow.keras.layers import Input, Dense, LSTM

from tensorflow.keras.models import Model

inputs = Input(shape=(50,1))

lstm_layer = LSTM(10)(inputs)

x = Dense(10, activation='relu')(lstm_layer)

output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=output)

### 6) 다르게 보이지만 동일한 표기

케라스의 Functional API가 익숙하지 않은 상태에서 Functional API를 사용한 코드를 보다가 혼동할 수 있는 점이 한 가지 있습니다. 바로 동일한 의미를 가지지만, 하나의 줄로 표현할 수 있는 코드를 두 개의 줄로 표현한 경우입니다.

result = Dense(128)(input)

위 코드는 아래와 같이 두 개의 줄로 표현할 수 있습니다.

dense = Dense(128)

result = dense(input)

